In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,017 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-se

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-17 21:14:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.66MB/s    in 0.2s    

2023-03-17 21:14:11 (5.66 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [6]:
# Get the number of rows in the DataFrame.
df.count()

3093869

# Transform the Data

## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df[["review_id", "customer_id", "product_id", "product_parent", "review_date"]]
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306| 2015-08-31|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678| 2015-08-31|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099| 2015-08-31|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257| 2015-08-31|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188| 2015-08-31|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982| 2015-08-31|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953| 2015-08-31|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933| 2015-08-31|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886| 2015-08-31|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381| 2015-08-31|
|R1DGA6UQIVLKZ7|    2781942|B007B5V092|     152891509| 2015-08-31|
| RLQT3V8SMNIBH|     707292|B00IODHGVG|     717335277| 2015-08

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df[["product_id", "product_title"]].drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JL2498C|Jensen JTA-410-BL...|
|B00OUKG3NE|Sennheiser Androi...|
|B0083H9R6A|Electrified ELPLP...|
|B0097BEE9Q|Apple iPod nano 1...|
|B0037Z4MCM|Pyle Marine Sport...|
|B00P0BQKBC|Water & Wood Lock...|
|B00TP1C1UC|AmazonBasics 6-Ou...|
|B005AIF0X4|iHip MVF1030WO Ma...|
|B009JJKZSK|Click 'N Dig! Key...|
|B00KXDS4VQ|enKo products HDM...|
|B009W857SG|TETC Cable (3 Fee...|
|B007LG453Y|Tmvel TVM-UK2 Uni...|
|B00KFSO0SA|Monoprice 11149 3...|
|B00IH5U050|Digital Army Camo...|
|B00JDYMPYS|Sylvania SKCR2713...|
|B00YDMW0DQ|Sound Intone HD30...|
|B00AR0VCXW|Pro-Ject Speed Bo...|
|B006BE6WK6|12 PIECE Ear Hook...|
|B00TM1CT98|Bose Acoustimass ...|
|B00P8D911I|AuraBeam Samsung ...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
#customers_df.orderBy(desc("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|             6|
|   28871066|             1|
|    6711334|             2|
|   46393060|             1|
|   48101124|             1|
|   38209321|             1|
|   47108763|             4|
|   43252205|             1|
|   41283385|             1|
|   46909180|             5|
|    5453476|             2|
|   12406466|             3|
|   48146680|             1|
|    5953456|             1|
|   16457323|             5|
|   24540309|             1|
|   43366878|             2|
|   28796717|             1|
|   28258386|             1|
|   26079415|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [10]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df[["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]]
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            

# Load

In [11]:
mode = "append"
jdbc_url="jdbc:postgresql://utsa-22.cte2cynaqbpd.us-east-2.rds.amazonaws.com:5432/utsa_22_big_data"
config = {"user":"postgres", "password": "postgres", "driver":"org.postgresql.Driver"}

In [12]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="electronics_review_id", mode=mode, properties=config)

In [13]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table="electronics_products", mode=mode, properties=config)

In [14]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table="electronics_customers", mode=mode, properties=config)

In [15]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table="electronics_vine", mode=mode, properties=config)